In [2]:

from bs4 import BeautifulSoup
import bs4
import requests
import time
import pandas as pd
import random
import numpy as np

In [3]:
# A "data junior" search seems to give me jobs that I think are interesting: 
# https://ca.indeed.com/jobs?q=data%20junior&l=Canada
# When you go on the next page it:s
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=10
# Then next page:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=20

# There only seems to be about 10 pages of results.
# If you try something like:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=1000
# It just give you the last page, so just check until the response stops changing?

# Also, you can use start=0 for the first page:
# https://ca.indeed.com/jobs?q=data+junior&l=Canada&start=0
# Is the same as
# https://ca.indeed.com/jobs?q=data%20junior&l=Canada

MAX_NUMBER_OF_PAGES_PER_SEARCH = 10 # testing

pages = []
for start in range(0, MAX_NUMBER_OF_PAGES_PER_SEARCH*10, 10):
    page = requests.get('https://ca.indeed.com/jobs?q=data%20junior&l=Canada&start=' + str(start))
    soup = BeautifulSoup(page.text, 'html.parser')
    pages.append(soup)
    time.sleep(random.randrange(3, 6) + random.random())

In [4]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# driver = webdriver.Chrome()
# driver.set_page_load_timeout(5)
# driver.get('https://ca.indeed.com/jobs?q=data%20junior&l=Canada&start=0')
# html = driver.page_source
# soup = BeautifulSoup(html)

# # check out the docs for the kinds of things you can do with 'find_all'
# # this (untested) snippet should find tags with a specific class ID
# # see: http://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class
# for tag in soup.find_all("div", class_="jobsearch-SerpJobCard unifiedRow row result"):
#     print(tag.text)

In [5]:
# When you search there's box cards for each posting, the class for the box is:
# class="jobsearch-SerpJobCard unifiedRow row result clickcard"
# that wasn't working for me though, so I changed it to 'class':"job_seen_beacon"


titles = pd.Series([], dtype='string')
companyNames = pd.Series([], dtype='string')
jobSnippets = pd.Series([], dtype='string')
for page in pages:
    jobs = page.find_all('div', attrs={'class':"job_seen_beacon"})
    for job in jobs:
        
        # Job Title
        title = job.find('h2', attrs={'class':'jobTitle'}).find('span', attrs={'title':True}).next 
        titles = titles.append(pd.Series([str(title)]))

        # Company Name
        companyName = job.find('span', attrs={'class':'companyName'})
        tempCompanyName = companyName
        companyName = companyName.find('a', attrs={'class':'turnstileLink companyOverviewLink'})
        if companyName is None: companyName = tempCompanyName
        companyName = companyName.next       
        companyNames = companyNames.append(pd.Series([str(companyName)]))
        
        jobSnippet = job.find('div', attrs={'class':'job-snippet'}).next
        jobSnippets = jobSnippets.append(pd.Series([str(jobSnippet)]))

pd.set_option('display.max_colwidth', None)
titles, companyNames, jobSnippets 

(0                                    Junior Data Analyst
 0                                    Junior Data Analyst
 0                                    Junior Data Analyst
 0    Product Manager, Junior - Global Data - Remote, CAN
 0                                    Junior Data Analyst
                             ...                         
 0                                    Analyst, Business I
 0        Laboratory Assistant I - Virology Pre-analytics
 0                         Business Development Analyst I
 0                               Junior Financial Analyst
 0                   Data Steward I, ITSS Data Governance
 Length: 141, dtype: object,
 0                        O2E Brands
 0           OSL Retail Services Inc
 0    Scandinavian Building Services
 0        Flight Centre Travel Group
 0                         FlightHub
                   ...              
 0              Mosaic North America
 0    Alberta Precision Laboratories
 0                   Best Buy Canada


In [8]:
pd.set_option('display.max_rows', 500)
pd.DataFrame({f'{titles=}'.split('=')[0]: titles, f'{companyNames=}'.split('=')[0]: companyNames})

,titles,companyNames
0,Junior Data Analyst,O2E Brands
0,Junior Data Analyst,OSL Retail Services Inc
0,Junior Data Analyst,Scandinavian Building Services
0,"Product Manager, Junior - Global Data - Remote, CAN",Flight Centre Travel Group
0,Junior Data Analyst,FlightHub
0,Junior Research Analyst,Carleton University
0,Jr. Data Analyst,Ratehub
0,JUNIOR FINANCIAL ANALYST,Ministry of Colleges and Universities
0,Junior Data Engineer,Kora
0,Junior Machine Learning Scientist,Ecoation
